In [1]:
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import tifffile as tiff
import seaborn as sns
import pickle

import networkx as nx
from sklearn.model_selection import train_test_split

In [2]:
import torch_geometric.utils
import torch

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

In [6]:
import lightning.pytorch as pl
import PPIGraph

# Test custom torch dataset

In [7]:
from torch_geometric.loader import DataLoader


In [8]:
condition_mapping = {'HCC827Ctrl': 0, 'HCC827Osim': 1}


In [9]:
graph_path = data_dir / 'OCT Cell Culture' / 'Whole' / 'graphs' 

dataset = PPIGraph.GraphDataset(graph_path, 'raw', 'pt', condition_mapping=condition_mapping, n_c=2)
train_set, val_set, test_set = PPIGraph.train_test_val_split(dataset)

# Create Dataloader
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)


In [10]:
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

Dataset: GraphDataset(2064):
Number of graphs: 2064
Number of features: 5
Number of classes: 2


In [11]:
print(f'Train set: {len(train_set)}, test set: {len(test_set)}, val set: {len(val_set)}')

Train set: 992, test set: 825, val set: 247


In [12]:
for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()
    break

Step 1:
Number of graphs in the current batch: 32
DataBatch(edge_index=[2, 14760], pos=[2787, 2], labels=[2787, 5], nuclei=[2787], weight=[14760], condition=[32], fov=[32], id=[32], train_mask=[2787], test_mask=[2787], x=[2787, 5], y=[32], edge_weight=[14760], name=[32], batch=[2787], ptr=[33])



# Test model

## Manual train GCN compoenents

In [13]:
def train(model, loader, epochs, criterion, optimizer, device, mlp=True):
    model.train()
    for epoch in range(epochs):
        for step, data in enumerate(loader):
            data = data.to(device)
            optimizer.zero_grad()
            if mlp:
                input_data = data.x.to(device)
                out = model(input_data)
            else:
                x, edge_index, edge_weight = data.x, data.edge_index, data.edge_weight
                out = model(x, edge_index, edge_weight=edge_weight)
            loss = criterion(out, data.nuclei)
            loss.backward()
            optimizer.step()
        if epoch%10==0:
            print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
            
def test(model, loader, device, mlp=True):
    model.eval()
    test_accs = [] 

    for step, data in enumerate(test_loader):
        data = data.to(device)
        if mlp:
            input_data = data.x.to(device)
            out = model(input_data)
        else:
            x, edge_index, edge_weight = data.x, data.edge_index, data.edge_weight
            out = model(x, edge_index, edge_weight=edge_weight)
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        test_correct = pred == data.nuclei  # Check against ground-truth labels.
        test_acc = int(test_correct.sum()) / len(pred)
        test_accs.append(test_acc)
        print(test_acc)
        
    return test_accs 

In [14]:
AVAIL_GPUS = [1]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [15]:
# %time
# # Create model, criterion and optimizer
# model_gcn = PPIGraph.GNNModel("GCN", 5, 16, 2).to(device)
# criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.
# optimizer = torch.optim.Adam(model_gcn.parameters(), lr=0.005, weight_decay=5e-4)

# # Train Loop
# train(model_gcn, train_loader, 10, criterion, optimizer, device, mlp=False)
# test_accs = test(model_gcn, test_loader, device, mlp=False)

## Lightning train

In [16]:
import itertools
from lightning.pytorch.accelerators import find_usable_cuda_devices
import wandb

In [17]:
condition = 'base'
checkpoint_folder = (Path().cwd().parents[0]).absolute() / 'data' / "saved_models" / f"Graph_GNNs_{condition}" 
project_name = f'PLA_042623_{condition}'

In [18]:
# AVAIL_GPUS = [1]
# BATCH_SIZE = 64 if AVAIL_GPUS else 32

# # Setting the seed
# pl.seed_everything(42)

# num_layers = [2,3,4]
# hiddens = [16, 32, 64]
# pools = ['mean', 'max', 'sum', 'attention', 'attention2']

# epochs = 30

In [19]:
# for NUM_LAYERS, HIDDEN_CHANNELS, pool in list(itertools.product(*[num_layers, hiddens, pools])):
    
#     # Path to the folder where the pretrained models are saved
#     CHECKPOINT_PATH = checkpoint_folder / f'{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot' / pool
#     CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)

#     # Run training
#     models = ['GCN']
#     for model_name in models:
#         run = wandb.init(project=project_name, name=model_name+f'_{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot')
#         model, result, trainer = PPIGraph.train_graph_classifier(model_name, 
#                                                                  train_set, 
#                                                                  val_set, 
#                                                                  test_set, 
#                                                                  dataset, 
#                                                                  CHECKPOINT_PATH, 
#                                                                  AVAIL_GPUS, 
#                                                                  in_channels=5,
#                                                                  hidden_channels=HIDDEN_CHANNELS, 
#                                                                  out_channels = HIDDEN_CHANNELS,
#                                                                  num_layers=NUM_LAYERS, epochs=epochs,
#                                                                  embedding=False,
#                                                                  graph_pooling=pool)
#         run.finish()

In [20]:
# for NUM_LAYERS, HIDDEN_CHANNELS, pool in list(itertools.product(*[num_layers, hiddens, pools])):
    
#     # Path to the folder where the pretrained models are saved
#     CHECKPOINT_PATH = checkpoint_folder / f'{NUM_LAYERS}_{HIDDEN_CHANNELS}_emb' / pool
#     CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)

#     # Run training
#     models = ['GCN']
#     for model_name in models:
#         run = wandb.init(project=project_name, name=model_name+f'_{NUM_LAYERS}_{HIDDEN_CHANNELS}_emb')
#         model, result, trainer = PPIGraph.train_graph_classifier(model_name, 
#                                                                  train_set, 
#                                                                  val_set, 
#                                                                  test_set, 
#                                                                  dataset, 
#                                                                  CHECKPOINT_PATH, 
#                                                                  AVAIL_GPUS, 
#                                                                  in_channels=HIDDEN_CHANNELS,
#                                                                  hidden_channels=HIDDEN_CHANNELS, 
#                                                                  out_channels = HIDDEN_CHANNELS,
#                                                                  num_layers=NUM_LAYERS, epochs=epochs,
#                                                                  embedding=True,
#                                                                  graph_pooling=pool)
#         run.finish()

# K Fold on  filter dataset 

In [21]:
from sklearn.model_selection import KFold, StratifiedKFold
from torch.utils.data import SubsetRandomSampler

In [22]:
# Filter out by maximum number of counts per cell
min_count = 20
max_count = 70

graph_path = data_dir / 'OCT Cell Culture' / 'Whole' / 'graphs' 

dataset = PPIGraph.GraphDataset(graph_path, 'raw', 'pt', condition_mapping=condition_mapping, n_c=2)

# Create Dataloader
loader = DataLoader(dataset, batch_size=1, shuffle=False)

# Get Indices
indices = []
for step, data in enumerate(loader):
    if len(data.x) <= min_count:
        continue 
    
    if (data.x.sum(axis=0) >= max_count).any():
        continue
    indices.append(step)
    
# Get subset dataset
dataset_filtered = dataset.index_select(indices)


In [23]:
len(dataset_filtered)

1522

In [24]:
condition = 'Kfold'
checkpoint_folder = (Path().cwd().parents[0]).absolute() / 'data' / "saved_models" / f"Graph_GNNs_{condition}" 
project_name = f'PLA_042923_{condition}'

In [25]:
AVAIL_GPUS = [1]

# Setting the seed
pl.seed_everything(42)

NUM_LAYERS = 2
HIDDEN_CHANNELS = 16
pools = ['mean', 'max', 'sum', 'attention', 'attention2']

epochs = 100

Global seed set to 42


In [26]:
# k_folds = 5
# kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# for fold, (train_ids, valid_ids) in enumerate(kfold.split(dataset_filtered)):
#     train_subset = dataset_filtered.index_select(train_ids.tolist())
#     val_subset = dataset_filtered.index_select(valid_ids.tolist())
    
#     for pool in pools:
#         # Path to the folder where the pretrained models are saved
#         CHECKPOINT_PATH = checkpoint_folder / f'{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot_{fold}' / pool
#         CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)
        
#         checkpoint = CHECKPOINT_PATH / "GraphLevelGCN" / "GraphLevelGCN.ckpt" 
#         if checkpoint.exists():
#             print(checkpoint)
            

In [27]:
### GCN 

# k_folds = 5
# kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# for fold, (train_ids, valid_ids) in enumerate(kfold.split(dataset_filtered)):
#     train_subset = dataset_filtered.index_select(train_ids.tolist())
#     val_subset = dataset_filtered.index_select(valid_ids.tolist())
    
#     for pool in pools:
#         # Path to the folder where the pretrained models are saved
#         CHECKPOINT_PATH = checkpoint_folder / f'{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot_{fold}' / pool
#         CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)
        
#         # Skip already trained kfold and pool
#         checkpoint = CHECKPOINT_PATH / "GraphLevelGCN" / "GraphLevelGCN.ckpt" 
#         if checkpoint.exists():
#             print(checkpoint)
#             continue
        
#         # Run training
#         run = wandb.init(project=project_name, name=f'GCN_{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot_{fold}', 
#                         group=f'GCN_{pool}')
#         PPIGraph.train_graph_classifier_kfold('GCN', 
#                                              train_subset, 
#                                              val_subset, 
#                                              dataset, 
#                                              CHECKPOINT_PATH, 
#                                              AVAIL_GPUS, 
#                                              in_channels=5,
#                                              hidden_channels=HIDDEN_CHANNELS, 
#                                              out_channels = HIDDEN_CHANNELS,
#                                              num_layers=NUM_LAYERS, 
#                                              epochs=epochs,
#                                              embedding=False,
#                                              graph_pooling=pool)
#         run.finish()


In [28]:
condition = 'Kfold'
checkpoint_folder = (Path().cwd().parents[0]).absolute() / 'data' / "saved_models" / f"MLP_{condition}" 
project_name = f'PLA_042923_{condition}'

In [29]:
AVAIL_GPUS = [1]

# Setting the seed
pl.seed_everything(42)

NUM_LAYERS = 2
HIDDEN_CHANNELS = 16
pools = ['mean', 'max', 'sum', 'attention', 'attention2']

epochs = 100

Global seed set to 42


In [30]:
### MLP

k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

for fold, (train_ids, valid_ids) in enumerate(kfold.split(dataset_filtered)):
    train_subset = dataset_filtered.index_select(train_ids.tolist())
    val_subset = dataset_filtered.index_select(valid_ids.tolist())
    
    for pool in pools:
        # Path to the folder where the pretrained models are saved
        CHECKPOINT_PATH = checkpoint_folder / f'{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot_{fold}' / pool
        CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)
        
        # Skip already trained kfold and pool
        checkpoint = CHECKPOINT_PATH / "GraphLevelMLP" / "GraphLevelMLP.ckpt" 
        if checkpoint.exists():
            print(checkpoint)
            continue
        
        # Run training
        run = wandb.init(project=project_name, name=f'MLP_{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot_{fold}', 
                        group=f'MLP_{pool}')
        PPIGraph.train_graph_classifier_kfold('MLP', 
                                             train_subset, 
                                             val_subset, 
                                             dataset, 
                                             CHECKPOINT_PATH, 
                                             AVAIL_GPUS, 
                                             in_channels=5,
                                             hidden_channels=HIDDEN_CHANNELS, 
                                             out_channels = HIDDEN_CHANNELS,
                                             num_layers=NUM_LAYERS, 
                                             epochs=epochs,
                                             batch_size=128,
                                             embedding=False,
                                             graph_pooling=pool)
        run.finish()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: thoomas. Use `wandb login --relogin` to force relogin


Global seed set to 42
C:\Users\thu71\Anaconda3\envs\snowflake\lib\site-packages\pytorch_lightning\loggers\wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42


Using MLP


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type             | Params
-------------------------------------------------
0 | x_embedding | Identity         | 0     
1 | model       | MLPModel         | 368   
2 | head        | Sequential       | 154   
3 | loss_module | CrossEntropyLoss | 0     
4 | train_acc   | BinaryAccuracy   | 0     
5 | train_auroc | BinaryAUROC      | 0     
6 | train_f1    | BinaryF1Score    | 0     
7 | valid_acc   | BinaryAccuracy   | 0     
8 | valid_auroc | BinaryAUROC      | 0     
9 | valid_f1    | BinaryF1Score    | 0     
-------------------------------------------------
522       Trainable params
0         Non-trainable params
522       Total params
0.002     Total estimated model params size (MB)
C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:478: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shu

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▅▅▆▆▇▇██▇█▇█▇▇█████▇▇█▇▇███▇██▇▇▇█▇▇▇██
train_auc,▁▁▂▂▃▆▆▇▇▆▇▇▇▇▇▇▇█▇▇▆▆█▅▆▇▇▇▇▇▇▆▆▆▇▇▆▇▇▇
train_f1,▆▂▁▁▄▆▇▇▇▆█▇▇█▇▇▇█▇▇▇▇█▆▇▇▇▇▇▇▇▇▇▆▇▇▆█▇█
train_loss_epoch,█▆▆▆▄▃▂▂▂▂▂▃▂▂▂▁▃▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▁▂▂▃▁▂
train_loss_step,█▇▇▅▄▃▄▄▆▃▆▇▄▄▄▂▃▄▅▃▄▂▄▃▄▁▃▃▅▄▃▃▇▁▃█▂▄▃▅
trainer/global_step,▁▁▁▂▂▁▂▂▁▃▃▂▃▃▂▄▄▂▂▄▂▂▅▅▂▆▆▂▆▆▃▇▇▃▃▇▃▃█▃
val_acc,▁▁▁▁▄▇▇▇▇▇█▇▇▇▇█▇▇▇▇▇█▇██▇▇▇█▇▇█▇█▇▇▇▇▇▇
val_auc,▁▁▁▁▃██▇█▇███▇███████▇█▇▇████████▇██▇█▇█
val_f1,▁▁▁▁▄████▇███████████▇█▇▇████████▇██▇███


...

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇█▇█▇▇▇▇▇█▇▇█▇
train_auc,▁▂▂▂▄▅▆▆▆▆▆▇▆▆▇▆▆▆▇▆▇▇▇▇█▇▆▇▆█▇▇▆▆▇▇▇▇▇▆
train_f1,▆▂▁▁▄▆▇▆▇▇▇█▇▇█▇▆▆█▆▇▇▇▇█▇▆▇▇█▇▇▇▇██▇▇█▇
train_loss_epoch,█▆▆▅▃▃▂▂▂▂▂▂▂▁▂▂▂▁▂▂▂▁▁▂▁▂▂▂▁▂▂▂▂▂▁▂▂▁▁▂
train_loss_step,█▇█▅▅▅▆▄▅▅▇▅▄▅▅▅▆▄▅▇▄▄▃▅▁▅▄▇▄▆▅▆▄▅▅▅▆▄▃▆
trainer/global_step,▁▁▁▂▂▁▂▂▁▃▃▂▃▃▂▄▄▂▂▄▂▂▅▅▂▆▆▂▆▆▃▇▇▃▃▇▃▃█▃
val_acc,▁▁▁▁▅▆▇██▆██▆▇▇▇▇█▆▇▆▆▄▆▆▆▇▆▆▄▆▆█▆▆▆▆▄▆▆
val_auc,▁▁▁▁▅▆▆█▇▆▇▇▆▆▇▆▇▇▅▆▆▆▃▆▆▆▆▆▆▄▆▆▇▆▆▆▆▄▆▆
val_f1,▁▁▁▁▅▇▆█▇▆▇▇▆▇▇▇▇▇▆▇▆▆▄▆▆▆▇▆▆▅▆▆▇▆▆▆▆▅▆▆


Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

   | Name        | Type             | Params
--------------------------------------------------
0  | x_embedding | Identity         | 0     
1  | model       | MLPModel         | 368   
2  | head        | Sequential       | 154   
3  | loss_module | CrossEntropyLoss | 0     
4  | train_acc   | BinaryAccuracy   | 0     
5  | train_auroc | BinaryAUROC      | 0     
6  | train_f1    | BinaryF1Score    | 0     
7  | valid_acc   | BinaryAccuracy   | 0     
8  | valid_auroc | BinaryAUROC      | 0     
9  | valid_f1    | BinaryF1Score    | 0     
10 | pool        | Attention_module | 281   
--------------------------------------------------
803       Trainable params
0         Non-trainable params
803       Total params
0.003     Total estimated model para

Using MLP


`Trainer.fit` stopped: `max_epochs=100` reached.


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▅▆▆▇▇███▇▇▇█▇█▇█▇▇█▇▇███▇▇█████████▇███
train_auc,▁▁▁▂▅▇▇▇▇▇▆▇▇▇█▇▇▅▆▆▇▇█▇▇▆▆▇▇██▇▇▇██▆▇██
train_f1,▇▂▁▁▆█▇▇▇▇▇▇▇██▇▇▆▆▇▇▇█▇▇▆▇▇▇███▇▇██▇▇██
train_loss_epoch,█▆▆▅▃▂▂▁▁▂▁▂▁▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▂▁▁▂▁▁▁▁▂▁▁▁
train_loss_step,█▇▅▆▃▂█▃▁▄▁▃▂▅▄▄▄▅▁▂▃▅▂▃▃▁▃▃▄▆▅▄▂▃▃▅▆▃▃▃
trainer/global_step,▁▁▁▂▂▁▂▂▁▃▃▂▃▃▂▄▄▂▂▄▂▂▅▅▂▆▆▂▆▆▃▇▇▃▃▇▃▃█▃
val_acc,▁▁▁▁▆▆██▆▇███▆█▇▇▇▇██▇▇██▇▇███▇█▇█▇█▇▆█▆
val_auc,▁▁▁▁▆▆██▆▇███▆█▇▇▇▇██▇▇███▇███▇█▇█▇█▇▆█▆
val_f1,▁▁▁▁▆▇██▇▇███▇█▇▇▇▇██▇▇███▇███▇█▇█▇█▇▇█▇
